In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from myxtts.config.config import XTTSConfig,ModelConfig,DataConfig,TrainingConfig
from myxtts import get_xtts_model, get_trainer, get_inference_engine

m=ModelConfig(
    text_encoder_dim=256,
    decoder_dim=512,
    n_mels=80,
    use_voice_conditioning=True
)
t=TrainingConfig(
    epochs=200,
    learning_rate=5e-5,
    optimizer="adamw",
    warmup_steps=2000,
)
d=DataConfig(
    batch_size=8,
    metadata_train_file="./data/train_metadata.csv",
    metadata_eval_file="./data/eval_metadata.csv",
    sample_rate=22050,
    normalize_audio=True,
)
# Load configuration
config = XTTSConfig(
    data=m,
    training=t, 
    data_config=d
)
#config = XTTSConfig.from_yaml("config.yaml")



ValueError: Unknown parameter: data_config. Valid parameters are: ['add_blank', 'audio_encoder_dim', 'audio_encoder_heads', 'audio_encoder_layers', 'batch_size', 'beta1', 'beta2', 'checkpoint_dir', 'dataset_name', 'dataset_path', 'decoder_dim', 'decoder_heads', 'decoder_layers', 'duration_loss_weight', 'epochs', 'eps', 'gradient_clip_norm', 'hop_length', 'kl_loss_weight', 'language', 'languages', 'learning_rate', 'log_step', 'max_audio_length', 'max_text_length', 'mel_loss_weight', 'metadata_eval_file', 'metadata_train_file', 'min_audio_length', 'n_fft', 'n_mels', 'normalize_audio', 'num_workers', 'optimizer', 'reference_audio_length', 'sample_rate', 'save_step', 'scheduler', 'scheduler_params', 'speaker_embedding_dim', 'text_cleaners', 'text_encoder_dim', 'text_encoder_heads', 'text_encoder_layers', 'text_vocab_size', 'tokenizer_model', 'tokenizer_type', 'train_split', 'trim_silence', 'use_voice_conditioning', 'use_wandb', 'val_split', 'val_step', 'wandb_project', 'warmup_steps', 'wav

In [ ]:
# Training
model = get_xtts_model()(config.model)
trainer = get_trainer()(config, model)
train_dataset, val_dataset = trainer.prepare_datasets("./data/ljspeech")
trainer.train(train_dataset, val_dataset)

# Inference
inference = get_inference_engine()(config, checkpoint_path="./checkpoints/best")
result = inference.synthesize("Hello world!")
inference.save_audio(result["audio"], "output.wav")